<a href="https://colab.research.google.com/github/windyday0622/windyday/blob/main/m7_NLP%20%EC%9D%91%EC%9A%A9/%20nlp_gpt_generation_train_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPT(Generative Pre-trained Transformer) 모델의 전체 과정을 설명하기 위해, 토크나이저 사용부터 모델링, 평가, 추론에 이르기까지의 단계별로 설명하겠습니다. GPT는 주로 자연어 생성에 사용되는 언어 모델로, 문맥을 이해하고 새로운 텍스트를 생성하는 데 탁월한 성능을 보입니다.

### 1. **토크나이저 사용**
토크나이저는 텍스트를 처리하기 위한 첫 번째 단계입니다. GPT 모델의 경우 **Byte-Pair Encoding (BPE)** 방식의 서브워드 기반 토크나이저를 주로 사용합니다. 텍스트를 모델이 이해할 수 있는 형식(토큰)으로 변환한 후, 해당 토큰을 숫자 ID로 매핑하는 작업을 수행합니다.

#### 1.1 **텍스트 입력 토크나이징**
GPT에서 입력 텍스트는 서브워드 수준에서 분리됩니다. 예를 들어, "The quick brown fox jumps over the lazy dog"이라는 문장이 입력되면, 토크나이저가 이 문장을 서브워드 단위로 분리하고, 각 서브워드에 해당하는 고유한 정수 ID로 변환합니다.

```python
from transformers import GPT2Tokenizer

# GPT 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 문장 토크나이징
input_text = "The quick brown fox jumps over the lazy dog"
tokens = tokenizer.tokenize(input_text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(tokens)     # ['The', 'Ġquick', 'Ġbrown', 'Ġfox', 'Ġjumps', 'Ġover', 'Ġthe', 'Ġlazy', 'Ġdog']
print(token_ids)  # [464, 1560, 635, 987, 1599, 544, 464, 17239, 3290]
```
GPT-2 모델의 vocab 사전에는 총 50,257개의 토큰이 포함되어 있습니다. 따라서, ID의 수는 0부터 50,256까지 총 50,257개입니다. 이 숫자는 GPT-2 모델이 사용하는 BPE (Byte Pair Encoding) 토크나이저에서 생성된 모든 토큰을 포함합니다.

이러한 토큰화 과정은 텍스트를 서브워드 단위로 쪼개고, 각 서브워드에 고유한 ID를 부여함으로써 모델이 텍스트 데이터를 수치화된 형태로 처리할 수 있도록 돕습니다.

#### 1.2 **모델 입력 생성**
토큰화된 텍스트는 모델에 입력으로 제공되기 위해 시퀀스 길이를 맞추기 위해 패딩이 적용되거나, 문장 시작을 알리는 `<|endoftext|>` 같은 특별 토큰이 추가될 수 있습니다.

### 2. **모델링**
GPT는 **Transformer의 디코더(decoder)** 아키텍처만을 사용하여, 주어진 시퀀스에서 다음 단어를 예측하는 방식으로 작동합니다. GPT의 핵심은 **자기 주의 메커니즘(self-attention mechanism)**을 통해 문맥 정보를 학습하는 것입니다.

#### 2.1 **모델 구조**
GPT 모델은 여러 개의 디코더 레이어로 구성되며, 각 레이어는 자기 주의 메커니즘과 피드포워드 뉴럴 네트워크로 이루어져 있습니다. 자기 주의 메커니즘을 통해 각 단어가 문맥 내 다른 단어들과의 관계를 학습하게 됩니다.

- **Self-Attention**: 문장 내의 각 단어가 문맥에서 다른 단어들과 어떻게 연관되는지를 학습합니다.
- **Layer Normalization**: 학습을 안정화하고, 성능을 향상시키기 위해 레이어 별로 입력을 정규화합니다.
- **Feedforward Neural Network**: Self-Attention을 통과한 데이터를 처리하여 모델의 출력을 생성합니다.

#### 2.2 **학습 (Pre-training)**
GPT 모델은 대량의 텍스트 데이터를 사용하여 사전 학습(pre-training)됩니다. 이 과정에서 모델은 주어진 문맥에서 다음 단어를 예측하는 **자기 지도 학습(self-supervised learning)**을 수행합니다.

- 입력 시퀀스에서 이전 단어들을 기반으로, 다음에 나올 단어를 예측합니다.
- **손실 함수**는 Cross-Entropy Loss를 사용하여 예측된 단어와 실제 단어 간의 차이를 측정합니다.
- 이 과정에서 모델은 문장의 구조, 문맥 및 언어 패턴을 학습합니다.

### 3. **평가 (Evaluation)**
모델을 평가하기 위한 과정에서는 보통 **Perplexity**라는 지표가 많이 사용됩니다. Perplexity는 언어 모델이 주어진 단어 시퀀스를 얼마나 잘 예측했는지를 나타내는 지표입니다. 더 구체적으로, 이는 모델이 예측한 확률 분포의 "확신도"를 나타냅니다. Perplexity가 낮을수록, 모델이 높은 확신을 가지고 올바르게 단어를 예측했다는 뜻입니다. 반대로, Perplexity가 높으면, 모델이 예측에 대해 확신이 낮았거나 틀린 예측을 많이 했다는 의미입니다.

- **Perplexity**: $PP(W) = exp \left( - \frac{1}{N} \sum_{i=1}^{N} \log P(w_i|w_{1:i-1}) \right)$
  - N은 문장의 단어 개수이고, $P(w_i|w_{1:i-1})$는 이전 단어들을 기반으로 모델이 예측한 다음 단어의 확률입니다.

### 4. **추론 (Inference)**
GPT 모델은 주로 **언어 생성** 또는 **다음 단어 예측** 작업에 사용됩니다. 추론 단계에서, 모델은 주어진 문장을 기반으로 다음에 나올 단어를 생성하며, 이 과정을 반복하여 원하는 길이의 텍스트를 생성합니다.

#### 4.1 **Greedy Search**
가장 확률이 높은 단어를 매번 선택하여 시퀀스를 생성하는 방식입니다. 빠르지만, 다양성 없이 항상 가장 가능성이 높은 단어만을 선택하므로 문장의 창의성이 떨어질 수 있습니다.

#### 4.2 **Beam Search**
여러 개의 경로를 동시에 탐색하며, 최적의 시퀀스를 찾는 방식입니다. 더 창의적이고 다양한 문장을 생성할 수 있지만, 계산 비용이 높을 수 있습니다.

#### 4.3 **Top-k Sampling 및 Nucleus Sampling**
모델이 예측한 단어 확률 분포에서 상위 k개의 단어만 고려하는 방식이 **Top-k Sampling**이며, 확률 누적 합이 일정 값(p)에 도달할 때까지 단어들을 고려하는 방식이 **Nucleus Sampling (Top-p Sampling)**입니다. 이 방식은 좀 더 다양하고 자연스러운 문장을 생성할 수 있습니다.

```python
from transformers import GPT2LMHeadModel

# GPT2 모델과 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 입력 텍스트
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 모델로 텍스트 생성
output = model.generate(input_ids, max_length=50, num_return_sequences=1, top_k=50, top_p=0.95)

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)
```

### 5. **최종 요약**
1. **토크나이저 사용**: 텍스트를 서브워드로 분리하고, 이를 고유한 토큰 ID로 변환합니다.
2. **모델링**: Transformer의 디코더 아키텍처를 사용하여 주어진 문맥에서 다음 단어를 예측합니다.
3. **평가**: 주로 Perplexity를 사용하여 모델 성능을 평가합니다.
4. **추론**: 다양한 기법(Greedy, Beam Search, Top-k Sampling, Nucleus Sampling)을 통해 텍스트를 생성합니다.

이러한 단계들을 통해 GPT는 입력 문장을 이해하고, 그에 기반하여 의미 있고 자연스러운 텍스트를 생성할 수 있습니다.

In [ ]:
# 의존성 패키지 설치하기
%pip install pip==24.0.0 -q
%pip install ratsnlp -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 35.9 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.6.1 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

https://github.com/ratsgo/ratsnlp/blob/master/ratsnlp/nlpbook/generation/arguments.py

In [ ]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2", # 40GB 이상의 텍스트로 학습된 한국어 디코더(decoder) 언어모델
    downstream_corpus_name="nsmc",
    downstream_model_dir="/content/drive/MyDrive/kdt_240424/m7_nlp응용/checkpoint-generation1",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다. 로거는 프로그램이 실행되는 동안 발생하는 다양한 정보를 기록합니다. 이 정보에는 디버깅 정보, 오류 메시지, 경고, 정보성 메시지 등이 포함됩니다.



In [ ]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/kdt_240424/m7_nlp응용/checkpoint-generation1', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/kdt_240424/m7_nlp응용/checkpoint-generation1', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, le

# 말뭉치 다운로드
실습에 사용할 말뭉치(NSMC)를 다운로드합니다.

In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 126MB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:00, 54.9MB/s]


In [ ]:
!ls /content/Korpora/nsmc

ratings_test.txt  ratings_train.txt


In [ ]:
import pandas as pd

# NSMC 학습 데이터 파일 경로
file_path = '/content/Korpora/nsmc/ratings_train.txt'

# 데이터 파일 읽기 (탭으로 구분된 CSV 파일 형식)
data = pd.read_csv(file_path, sep='\t')

# 데이터의 첫 몇 줄을 출력
print(data.head())

         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1


In [ ]:
# NSMC 테스트 데이터 파일 경로
file_path = '/content/Korpora/nsmc/ratings_test.txt'

# 데이터 파일 읽기 (탭으로 구분된 CSV 파일 형식)
test_data = pd.read_csv(file_path, sep='\t')

# 데이터의 첫 몇 줄을 출력
print(test_data.head())

        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>", # end-of-sentence token
)
# eos_token: 문장 마지막에 붙이는 스페셜 토큰(end of sentence)으로 SK텔레콤이 모델을 프리트레인할 때 이렇게 지정했기 때문에 같은 방식으로 사용

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


# 학습데이터 구축
학습데이터를 만듭니다.

In [ ]:
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 14.997 s]
INFO:ratsnlp:tokenize sentences [took 14.997 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [ ]:
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 4.626 s]
INFO:ratsnlp:tokenize sentences [took 4.626 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>


# 모델 초기화
프리트레인이 완료된 GPT2 모델을 읽고, 문장 생성 모델을 초기화합니다.

huggingface/transformers<br>

https://github.com/huggingface/transformers/blob/v4.25.1/src/transformers/models/gpt2/modeling_gpt2.py#L945

In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

# 학습 준비
Task와 Trainer를 준비합니다.

In [ ]:
# https://github.com/ratsgo/ratsnlp/blob/master/ratsnlp/nlpbook/generation/task.py
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/lightning.py:2046: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook


In [ ]:
# ratsnlp/ratsnlp/nlpbook/trainer.py

trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-generation`)에 저장됩니다.

In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# 폴더 복사
!cp -r /gdrive/My Drive/nlpbook/checkpoint-generation2 /gdrive/MyDrive/kdt_231026/m7_nlp응용/gpt